In [ ]:
# default_exp dijsktra

# Dijsktra

> shortest path with weights

In [ ]:
# export
from queue import Queue
from typing import List, Dict
from collections import namedtuple

import pandas as pd

import graph_utils.core as gu
from graph_utils.shortest_path import shortest_path

In [ ]:
# export
from heapq import heapify, heappush, heappop

class priority_dict(dict):
    """Dictionary that can be used as a priority queue.

    Keys of the dictionary are items to be put into the queue, and values
    are their respective priorities. All dictionary methods work as expected.
    The advantage over a standard heapq-based priority queue is
    that priorities of items can be efficiently updated (amortized O(1))
    using code as 'thedict[item] = new_priority.'

    The 'smallest' method can be used to return the object with lowest
    priority, and 'pop_smallest' also removes it.

    The 'sorted_iter' method provides a destructive sorted iterator.
    
    SOURCE:
    https://code.activestate.com/recipes/522995-priority-dict-a-priority-queue-with-updatable-prio/
    """
    
    def __init__(self, *args, **kwargs):
        super(priority_dict, self).__init__(*args, **kwargs)
        self._rebuild_heap()

    def _rebuild_heap(self):
        self._heap = [(v, k) for k, v in self.items()]
        heapify(self._heap)

    def smallest(self):
        """Return the item with the lowest priority.

        Raises IndexError if the object is empty.
        """
        
        heap = self._heap
        v, k = heap[0]
        while k not in self or self[k] != v:
            heappop(heap)
            v, k = heap[0]
        return k

    def pop_smallest(self):
        """Return the item with the lowest priority and remove it.

        Raises IndexError if the object is empty.
        """
        
        heap = self._heap
        v, k = heappop(heap)
        while k not in self or self[k] != v:
            v, k = heappop(heap)
        del self[k]
        return k

    def __setitem__(self, key, val):
        # We are not going to remove the previous value from the heap,
        # since this would have a cost O(n).
        
        super(priority_dict, self).__setitem__(key, val)
        
        if len(self._heap) < 2 * len(self):
            heappush(self._heap, (val, key))
        else:
            # When the heap grows larger than 2 * len(self), we rebuild it
            # from scratch to avoid wasting too much memory.
            self._rebuild_heap()

    def setdefault(self, key, val):
        if key not in self:
            self[key] = val
            return val
        return self[key]

    def update(self, *args, **kwargs):
        # Reimplementing dict.update is tricky -- see e.g.
        # http://mail.python.org/pipermail/python-ideas/2007-May/000744.html
        # We just rebuild the heap from scratch after passing to super.
        
        super(priority_dict, self).update(*args, **kwargs)
        self._rebuild_heap()

    def sorted_iter(self):
        """Sorted iterator of the priority dictionary items.

        Beware: this will destroy elements as they are returned.
        """
        
        while self:
            yield self.pop_smallest()


In [ ]:
# export
Row = namedtuple('Node', ['distance_from_source', 'preceding_vertex'])

In [ ]:
# export
def build_distance_table(graph:gu.Graph, source:int):
    distance_table = dict()

    # initiate an empty distance table
    for i in range(graph.numVertices):
        #distance_table[i] = Row(distance_from_source=None, preceding_vertex=None)
        distance_table[i] = Row(None, None)

    # distance to the source from itself is 0
    distance_table[source] = Row(0, source)


    # Holds mapping of vertex id to distance
    # from source. Access the highest priority (lowest distance)
    # item first
    priority_queue = priority_dict()
    # priority_queue[vertex number] = distance

    priority_queue[source] = 0

    while priority_queue:
        current_vertex = priority_queue.pop_smallest()

        current_distance = distance_table[current_vertex][0]

        for neigbor in graph.get_adjacent_vertices(current_vertex):
            distance = current_distance + g.get_edge_weight(current_vertex, neigbor)

            neigbor_distance = distance_table[neigbor][0]

            if neigbor_distance is None or neigbor_distance > distance:
                distance_table[neigbor] = Row(distance, current_vertex)

                priority_queue[neigbor] = distance


    return distance_table

In [ ]:
g = gu.AdjacencyMatrixGraph(8, directed=False)
g.add_edge(0, 1, 1)
g.add_edge(1, 2, 2)
g.add_edge(1, 3, 6)
g.add_edge(2, 3, 2)
g.add_edge(1, 4, 3)
g.add_edge(3, 5, 1)
g.add_edge(5, 4, 5)
g.add_edge(3, 6, 1)
g.add_edge(6, 7, 1)
g.add_edge(0, 7, 8)

In [ ]:
assert pd.DataFrame(build_distance_table(g, 0)).to_dict() == {
 0: {0: 0  , 1: 0  },
 1: {0: 1.0, 1: 0.0},
 2: {0: 3.0, 1: 1.0},
 3: {0: 5.0, 1: 2.0},
 4: {0: 4.0, 1: 1.0},
 5: {0: 6.0, 1: 3.0},
 6: {0: 6.0, 1: 3.0},
 7: {0: 7.0, 1: 6.0}}

In [ ]:
assert shortest_path(g, 0, 6, build_distance_table) == [0, 1, 2, 3, 6]
assert shortest_path(g, 4, 7, build_distance_table) == [4,5,3,6,7]
assert shortest_path(g, 7, 0, build_distance_table) == [7,6,3,2,1,0]

In [ ]:
g = gu.AdjacencyMatrixGraph(8, directed=True)
g.add_edge(0, 1, 1)
g.add_edge(1, 2, 2)
g.add_edge(1, 3, 6)
g.add_edge(2, 3, 2)
g.add_edge(1, 4, 3)
g.add_edge(3, 5, 1)
g.add_edge(5, 4, 5)
g.add_edge(3, 6, 1)
g.add_edge(6, 7, 1)
g.add_edge(0, 7, 8)

In [ ]:
assert shortest_path(g, 0, 6, build_distance_table) == [0, 1, 2, 3, 6]
assert shortest_path(g, 4, 7, build_distance_table) is None
assert shortest_path(g, 7, 0, build_distance_table) is None

There is no path from 4 to 7
There is no path from 7 to 0
